In [1]:
# Install dependencies
!pip install matplotlib pandas seaborn scikit-learn openpyxl
!pip install biopython

# Required imports
from Bio import SeqIO
import gzip
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
from sklearn.metrics import roc_auc_score

# Set root path
os.chdir('/content')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.7 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/ArcInstitute/evo2.git

Cloning into 'evo2'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 262 (delta 117), reused 79 (delta 79), pack-reused 125 (from 1)
Receiving objects: 100% (262/262), 25.38 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [3]:
#!git clone --recurse-submodules git@github.com:ArcInstitute/evo2.git
!cd /content/evo2 && pip install .

Processing /content/evo2
  Preparing metadata (setup.py) ... done
  Created wheel for evo2: filename=evo2-0.1.0-py3-none-any.whl size=23038 sha256=434777c34dff0c43780fddd4a14ecb3bf6ba8c4de9a930e406dbe844f53b885d
  Stored in directory: /tmp/pip-ephem-wheel-cache-uz5ugpsg/wheels/ae/53/b1/21a3a4a9597426a9ffdde3e83412b611e75d3dc30787512fd8
Successfully built evo2


In [4]:
!pip install vtx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: einops
    Found existing installation: einops 0.8.0
    Uninstalling einops-0.8.0:
      Successfully uninstalled einops-0.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vortex 0.0.2 requires einops==0.8.0, but you have einops 0.8.1 which is incompatible.


In [5]:
# Step 1: Uninstall existing flash-attn (ignore errors if not installed)
!pip uninstall flash-attn -y

# Step 2: Clone the Flash-Attention repo
!git clone https://github.com/Dao-AILab/flash-attention.git

# Step 3: Change directory and install flash-attn with no build isolation
%cd flash-attention
!pip install flash-attn . --no-build-isolation

Cloning into 'flash-attention'...
remote: Enumerating objects: 10258, done.
remote: Counting objects: 100% (2900/2900), done.
remote: Compressing objects: 100% (276/276), done.
remote: Total 10258 (delta 2737), reused 2624 (delta 2624), pack-reused 7358 (from 2)
Receiving objects: 100% (10258/10258), 9.79 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (7927/7927), done.
/content/flash-attention
Processing /content/flash-attention
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 116.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.8.1-cp311-cp311-linux_x86_64.whl size=126076958 sha256=e7f0c4b5a0685c15e449a3db7f6efe25b955d6b153d1acdc3208b136738513a3
  Stored in directory: /root/.cache/pip/wheels/1d/53/c5/000e1a593bdcce30072c46e5cf710bb5b1e2fa0fded85ca9e4
Successfully built flash_attn


In [6]:
!pip install transformer_engine[pytorch]==1.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.0/460.0 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 4.8 MB/s eta 0:00:00
  Created wheel for transformer_engine_torch: filename=transformer_engine_torch-1.13.0-cp311-cp311-linux_x86_64.whl size=30259669 sha256=8d31e4f82f20e866896350856dd1f1efcff1bd775bd0247ef623892af09f6eca
  Stored in directory: /root/.cache/pip/wheels/0a/a4/a9/ffe65f45da165ca92fbd32267de0cbf654dc56221e3105af54
Successfully built transformer_engine_torch
  Attempting uninstall: transformer_engine_cu12
    Found existing installation: transformer_engine_cu12 2.5.0
    Uninstalling transformer_engine_cu12-2.5.0:
      Successfully uninstalled transformer_engine_cu12-2.5.0
  Attempting uninstall: transformer_engine_torch
    Found existing installation: transformer_engine_torch 2.5.0
    Uninstalling transformer_en

In [3]:
import pandas as pd
from evo2.models import Evo2
import csv

# Load input CSV
df = pd.read_csv('/content/full_constructsb.csv')

# Define columns
ref_col = 'Codon'
exclude_cols = ['Unnamed: 0', ref_col]
variant_cols = [col for col in df.columns if col not in exclude_cols]

# Reconstruct reference sequence
reference_seq = ''.join(df[ref_col])

# Load Evo2 model
model = Evo2("evo2_7b_base")

# Score reference sequence once
ref_score = model.score_sequences([reference_seq])[0]

# Open output CSV and write header
output_path = '/content/variant_scores.csv'
with open(output_path, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=[
        'variant_name', 'ref_seq', 'mut_seq', 'ref_score', 'mut_score', 'delta_score'
    ])
    writer.writeheader()

    # Score variants one by one
    for variant_col in variant_cols:
        variant_seq = ''.join(df[variant_col])
        try:
            mut_score = model.score_sequences([variant_seq])[0]
            writer.writerow({
                'variant_name': variant_col,
                'ref_seq': reference_seq,
                'mut_seq': variant_seq,
                'ref_score': ref_score,
                'mut_score': mut_score,
                'delta_score': mut_score - ref_score
            })
        except Exception as e:
            print(f"Error scoring {variant_col}: {e}")

Found existing merged file: /root/.cache/huggingface/evo2_40b_base.pt


  0%|          | 0/50 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 684.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 183.38 MiB is free. Process 380710 has 21.98 GiB memory in use. Of the allocated memory 19.73 GiB is allocated by PyTorch, and 2.04 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
from google.colab import files

# Save your dataframe to CSV
result_df.to_csv('/content/result_df.csv', index=False)

# Download the file to your local machine
files.download('/content/result_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>